# 데이터 전처리하기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
#날씨 .csv파일 불러오기
weather_df = pd.read_csv("./OBS_ASOS_DD_20230101194708.csv", encoding="euc-kr")
# 데이터의 최초 5개의 행 보여줌 
weather_df.head()

In [ ]:
# 특정 열 제거 (최초 2개열)
weather_df = weather_df.drop(columns=["지점", "지점명"])
# 열 이름 바꿈 
weather_df = weather_df.rename(columns={"일시": "년월일"})
weather_df = weather_df.set_index("년월일")

In [ ]:
# head: 상위 5개 행, info, describe, hist
weather_df.head()

In [ ]:
# 데이터에 대한 기본적인 정보
# 행수, 열수등, 어떤 데이터 형태인가?
weather_df.info()

In [ ]:
# 각 열에 대한 기본적인 통계, 평균등
weather_df.describe()

In [ ]:
# histogram
weather_df.hist()

In [ ]:
#확진자수 .csv파일 불러오기
covid_df = pd.read_csv("./인천광역시 남동구_일별 코로나19 확진자수 현황_20220928.csv", encoding="euc-kr")
covid_df.head()

In [ ]:
### 연습, covid_df의 통계, 히스토그램등 시각화 ###

In [ ]:
#.불필요한 열 제거, 
#'날짜'열을 '년월일'로 바꾸고 인덱스로 설정하기
covid_df = covid_df[["날짜", "확진자수"]]

covid_df = covid_df.rename(columns={"날짜": "년월일"})

covid_df = covid_df.set_index("년월일")

covid_df.head()

In [ ]:
#두 데이터 frame (weather dataframe, covid dataframe)합치기, 합쳐서 새로운 dataframe 만듬 
data_df = weather_df.merge(covid_df, left_index=True, right_index=True)
#NaN (Not-a-Number) 값 0으로 바꾸어주기
data_df = data_df.fillna(0)
# 두 데이터 프레임을 합친후에 상위 5개의 행 보여줌 
data_df.head()

In [ ]:
#.csv 파일로 저장하기
data_df.to_csv("weather_covid.csv")

# 시각화

In [ ]:
# 온도 변화 시각화 
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False
# 최초 100일 온도
df3 = weather_df.head(100)
x3 = df3.head(100).index
y3 = df3.head(100)["평균기온(°C)"]

In [ ]:
plt.figure(figsize=(20, 5))
plt.scatter(x3, y3)
plt.xticks(np.arange(0, len(x3), 7))
plt.xticks(rotation=30)
plt.title("인천 온도", fontsize=20)
plt.xlabel("년월일", fontsize=14)
plt.ylabel("온도", fontsize=14)
plt.show()

In [ ]:
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False
# 최초 100일 코로나 확진자수 dataframe
df1 = covid_df.head(100)

x2 = df1.index
y2 = df1.head(100)["확진자수"]

In [ ]:
plt.figure(figsize=(20, 5))
plt.scatter(x2, y2)
plt.xticks(np.arange(0, len(x2), 7))
plt.xticks(rotation=30)
plt.title("코로나19 일별 확진자수(인천)", fontsize=20)
plt.xlabel("년월일", fontsize=14)
plt.ylabel("확진자수", fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(df1.확진자수, bins=10)
plt.xticks(np.arange(0, max(df1.확진자수), 500))
plt.title("코로나19 일별 확진자수(인천)", fontsize=20)
plt.xlabel("계급(확진자수)", fontsize=14)
plt.ylabel("빈도수(일수)", fontsize=14)
plt.show()

# 선형회귀 모델 만들기

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LinearRegression

In [ ]:
#데이터 준비
xy = np.array(data_df, dtype=np.float32)
# 마지막 열 (확진자수)만 빼고 x_data 변수에 저장
x_data = xy[:, :-1]
# 마지막 열만 y_data에 저장 (예측하고 싶은 값 )
y_data = xy[:, -1:]

In [ ]:
# linear regression fitting
from sklearn.metrics import mean_squared_error
reg = LinearRegression()
reg.fit(x_data, y_data)

# fitting 결과
print(reg.coef_)
print(reg.intercept_)

# 어떤 특징이 가장 중요한가?
# 첫번째 특징, 즉, 평균 온도가 가장 확진자수에 영향을 많이 미침
# 단, 음의 상관관계 

# 평균 제곱 오차 출력
print(mean_squared_error(reg.predict(x_data), y_data, squared=False))

## 직접 입력을 넣어서 예측해보기

In [ ]:
temp = float(input("평균 기온: "))
rainfall = max(float(input("일강수량(mm): ")), 0)
max_windspeed = max(float(input("최대 순간 풍속 (m/s): ")), 0)
min_humidity = np.clip(float(input("최소 상대습도(%): ")), 0.0, 1.0)
snowfall = max(float(input("일 최심적설(cm): ")), 0)
print()

#입력 만들기
pred_x = [[temp, rainfall, max_windspeed, min_humidity, snowfall]]

n_covid = reg.predict(pred_x)[0][0]

if n_covid <= 0:
    print("확진자가 없을 것으로 예측됩니다.")
else:
    print(f"확진자 수가 약 {np.ceil(n_covid)}명으로 예측됩니다")

## 인구수 예측 (연습)

In [ ]:
import matplotlib.pyplot as plt
# 2010부터 2019까지 1씩 증가하면서 10개의 data 생성
X = np.linspace(2010, 2019, 10 ).reshape(-1, 1)
y=[470171, 471265, 484550, 436455, 435435, 438420, 406243, 355771, 326822, 303054]

# x축을 "연도", y축을 "출생아수"로 하는 산점도 그리기, fontsize=14



#---------------------------#

In [ ]:
# LinearRegression 모델 선언하기

#---------------------------#
# 모델 학습(fit)

#---------------------------#
print(reg.coef_) 
print(reg.intercept_)
# 산점도 그리기

#---------------------------#
# 예측, 즉, fitting한 결과 직선으로 그림 

#---------------------------#
plt.grid(True) 

In [ ]:
test_1 = np.array([[2020]])
# test_1에 대한 결과값을 예측하기
predicted = 
#---------------------------#
print(predicted)

In [ ]:
# ftting 결과 RMSE 출력하기

#---------------------------#